#**Quantitative Momentum Strategy**

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 10 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 10 stocks.

##**Library Imports**
The first thing we need to do is import the open-source software libraries that we'll be using in this project.

In [ ]:
!pip install numpy
!pip install pandas
!pip install requests
!pip install xlsxwriter
!pip install yfinance

In [2]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for storing output in excel
import math #The Python math module
from scipy import stats #The SciPy stats module
import yfinance as yf #TO get current rates for nifty 50
from datetime import datetime, timedelta #To foramt date time data

##**Importing Our List of Stocks**

The next thing we need to do is import the constituents of the Nifty 50.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

There's a static version of the Nifty 50 constituents available here.[ Click this link](https://drive.google.com/file/d/1S0AsigDz6g9CI0ed2vIERyX7mPXWJ90A/view?usp=sharing) to download them now. Upload this file in the colab so it can be accessed by other files in the script.

Now it's time to import these stocks to our colab file.

In [3]:
stocks = pd.read_csv('/content/nifty50_stocks.csv') #Read CSV file

In [4]:
temp = stocks.iloc[:,0] #dropping all extra columns
df = pd.DataFrame(temp)
df.columns

Index(['Ticker'], dtype='object')

#**Retrieving Data using Yahoo Finance**

To obtain the desired information from Yahoo Finance, we will use the yfinance library, which allows us to fetch data from Yahoo Finance's API. We are interested in retrieving the following details for each stock:




*   Price of each stock.
*   One-year price returns of each stock.

By utilizing the functionalities provided by yfinance, we can access and extract this data for analysis and further processing.


In [5]:
data =[]

# Loop over the stocks in the index
for symbol in df['Ticker']:

  symbol = symbol + '.NS'
  ticker = yf.Ticker(symbol)
  stock_info = ticker.info

  # Retrieve the current price data for each stock
  price = stock_info['currentPrice']

  # Get the one-year price returns
  returns = stock_info.get('52WeekChange') * 100 if stock_info.get('52WeekChange') else None

  # Append the data to the list
  data.append([symbol, price, returns, 'N/A'])


##**Adding Our Stocks Data to a Pandas DataFrame**
The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
# Create a DataFrame from the collected data
columns = ['Ticker', 'Stock Price', 'One-Year Price Return', 'Number Of Shares to Buy']
df_result = pd.DataFrame(data, columns=columns)

In [7]:
# Print the resulting DataFrame
print(df_result)

           Ticker  Stock Price  One-Year Price Return Number Of Shares to Buy
0   INDUSINDBK.NS      1294.00              65.050640                     N/A
1     AXISBANK.NS       977.55              55.276420                     N/A
2    POWERGRID.NS       245.20              11.971343                     N/A
3     ADANIENT.NS      2480.50              18.221319                     N/A
4           LT.NS      2364.15              54.144300                     N/A
5   ULTRACEMCO.NS      8392.05              56.002070                     N/A
6        CIPLA.NS       998.20               4.309475                     N/A
7       GRASIM.NS      1775.85              35.243762                     N/A
8   TATAMOTORS.NS       570.75              43.071640                     N/A
9    BRITANNIA.NS      4958.85              45.708466                     N/A
10        NTPC.NS       186.35              29.660726                     N/A
11     DRREDDY.NS      4825.05               9.667635           

# **Removing Low-Momentum Stocks**
The investment strategy that we're building seeks to identify the 10 highest-momentum stocks in the Nifty 50.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 10.

In [9]:
df_result.sort_values('One-Year Price Return', ascending = False, inplace = True)
df_result = df_result[:10]
df_result.reset_index(drop = True, inplace = True)
df_result

,Ticker,Stock Price,One-Year Price Return,Number Of Shares to Buy
0,ITC.NS,447.45,70.847714,N/A
1,INDUSINDBK.NS,1294.00,65.050640,N/A
2,ULTRACEMCO.NS,8392.05,56.002070,N/A
3,AXISBANK.NS,977.55,55.276420,N/A
4,LT.NS,2364.15,54.144300,N/A
5,BRITANNIA.NS,4958.85,45.708466,N/A
6,TATAMOTORS.NS,570.75,43.071640,N/A
7,TITAN.NS,2907.00,41.318320,N/A
8,ICICIBANK.NS,930.15,39.064110,N/A
9,JSWSTEEL.NS,772.00,38.618672,N/A


# **Calculating the Number of Shares to Buy**
We now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:478768698
478768698


In [11]:
position_size = float(portfolio_size) / len(df_result.index)
for i in range(0, len(df_result['Ticker'])):
    df_result.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / df_result['Stock Price'][i])
df_result



,Ticker,Stock Price,One-Year Price Return,Number Of Shares to Buy
0,ITC.NS,447.45,70.847714,106999
1,INDUSINDBK.NS,1294.00,65.050640,36999
2,ULTRACEMCO.NS,8392.05,56.002070,5705
3,AXISBANK.NS,977.55,55.276420,48976
4,LT.NS,2364.15,54.144300,20251
5,BRITANNIA.NS,4958.85,45.708466,9654
6,TATAMOTORS.NS,570.75,43.071640,83884
7,TITAN.NS,2907.00,41.318320,16469
8,ICICIBANK.NS,930.15,39.064110,51472
9,JSWSTEEL.NS,772.00,38.618672,62016


## **Building a Better (and More Realistic) Momentum Strategy**

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

High-quality momentum stocks show "slow and steady" outperformance over long periods of time
Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.
The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:


*   1-month price returns

*   3-month price returns
*   6-month price returns


*   1-year price returns






Let's start by building our DataFrame. You'll notice that I use the abbreviation hqm often. It stands for high-quality momentum.

In [12]:
hqm_columns = [
                'Ticker',
                'Price',
                'Number of Shares to Buy',
                'One-Year Price Return',
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]


hqm_dataframe = pd.DataFrame(columns = hqm_columns)


for symbol in df['Ticker']:
    symbol += '.NS'
    ticker = yf.Ticker(symbol)
    stock_info = ticker.info

    # Retrieve the current price data for each stock
    price = stock_info.get('currentPrice')

    # Calculate the price returns for different time periods
    end = pd.Timestamp.now()
    start_1y = end - pd.DateOffset(years=1)
    start_6m = end - pd.DateOffset(months=6)
    start_3m = end - pd.DateOffset(months=3)
    start_1m = end - pd.DateOffset(months=1)

    hist_1y = ticker.history(start=start_1y, end=end)
    hist_6m = ticker.history(start=start_6m, end=end)
    hist_3m = ticker.history(start=start_3m, end=end)
    hist_1m = ticker.history(start=start_1m, end=end)

    returns_1y = (hist_1y['Close'][-1] - hist_1y['Close'][0]) / hist_1y['Close'][0] * 100
    returns_6m = (hist_6m['Close'][-1] - hist_6m['Close'][0]) / hist_6m['Close'][0] * 100
    returns_3m = (hist_3m['Close'][-1] - hist_3m['Close'][0]) / hist_3m['Close'][0] * 100
    returns_1m = (hist_1m['Close'][-1] - hist_1m['Close'][0]) / hist_1m['Close'][0] * 100

    # Append the data to the dataframe
    data = pd.DataFrame([[symbol, price, 'N/A', returns_1y, 'N/A', returns_6m, 'N/A', returns_3m, 'N/A', returns_1m, 'N/A', 'N/A']], columns=hqm_columns)
    hqm_dataframe = pd.concat([hqm_dataframe, data], ignore_index=True)

hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

# **Calculating Momentum Percentiles**
We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

`One-Year Price Return`

`Six-Month Price Return`

`Three-Month Price Return`

`One-Month Price Return`





Here's how we'll do this:

In [13]:
# Calculate the percentile scores for each time period

time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for time_period in time_periods:
    column_name = f'{time_period} Price Return'
    percentile_column = f'{time_period} Return Percentile'
    hqm_dataframe[percentile_column] = hqm_dataframe[column_name].apply(
        lambda x: stats.percentileofscore(hqm_dataframe[column_name], x) / 100
    )

# Print each percentile score
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

hqm_dataframe

0     0.96
1     0.88
2     0.32
3     0.28
4     0.90
5     0.94
6     0.14
7     0.72
8     0.82
9     0.92
10    0.64
11    0.24
12    0.60
13    0.86
14    0.80
15    0.84
16    0.52
17    0.56
18    0.78
19    0.70
20    0.74
21    0.44
22    0.54
23    0.42
24    0.46
25    0.48
26    0.18
27    0.50
28    0.02
29    0.20
30    0.76
31    0.10
32    0.34
33    0.38
34    0.16
35    0.22
36    0.26
37    0.98
38    0.30
39    0.04
40    0.68
41    0.40
42    0.58
43    0.06
44    0.62
45    1.00
46    0.66
47    0.08
48    0.12
49    0.36
Name: One-Year Return Percentile, dtype: float64
0     0.66
1     0.52
2     0.92
3     0.02
4     0.72
5     0.94
6     0.10
7     0.26
8     1.00
9     0.86
10    0.82
11    0.54
12    0.62
13    0.50
14    0.46
15    0.88
16    0.24
17    0.36
18    0.90
19    0.44
20    0.68
21    0.22
22    0.14
23    0.60
24    0.30
25    0.80
26    0.04
27    0.56
28    0.34
29    0.28
30    0.64
31    0.20
32    0.74
33    0.12
34    0.32
35    0.58
36   

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,INDUSINDBK.NS,1294.75,N/A,59.913329,0.96,7.919223,0.66,28.095639,0.96,9.693465,0.96,N/A
1,AXISBANK.NS,977.45,N/A,50.528549,0.88,4.215742,0.52,18.645716,0.74,6.759120,0.82,N/A
2,POWERGRID.NS,245.20,N/A,17.385806,0.32,16.624812,0.92,7.971965,0.32,1.336074,0.24,N/A
3,ADANIENT.NS,2485.10,N/A,12.523769,0.28,-39.235860,0.02,33.613372,0.98,28.240382,1.00,N/A
4,LT.NS,2364.35,N/A,52.863701,0.90,8.151363,0.72,8.196088,0.34,4.815628,0.62,N/A
5,ULTRACEMCO.NS,8385.05,N/A,54.126322,0.94,17.492514,0.94,17.710098,0.68,7.696862,0.88,N/A
6,CIPLA.NS,998.20,N/A,1.512667,0.14,-11.172629,0.10,11.475596,0.40,6.057653,0.70,N/A
7,GRASIM.NS,1773.70,N/A,34.030946,0.72,-0.422522,0.26,14.752354,0.58,2.653821,0.30,N/A
8,TATAMOTORS.NS,570.95,N/A,37.720352,0.82,36.861044,1.00,38.641055,1.00,7.431480,0.86,N/A
9,BRITANNIA.NS,4959.00,N/A,53.399569,0.92,12.561500,0.86,18.460032,0.72,6.711506,0.80,N/A


## **Calculating the HQM Score**
We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic `mean`, we will use the mean function from Python's built-in `statistics` module.

In [14]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## **Selecting the 50 Best Momentum Stocks**
As before, we can identify the 10 best momentum stocks in our universe by sorting the DataFrame on the HQM Score column and dropping all but the top 10 entries.

In [15]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:11]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,TATAMOTORS.NS,570.95,N/A,37.720352,0.82,36.861044,1.00,38.641055,1.00,7.431480,0.86,0.92
1,INDUSINDBK.NS,1294.75,N/A,59.913329,0.96,7.919223,0.66,28.095639,0.96,9.693465,0.96,0.885
2,ULTRACEMCO.NS,8385.05,N/A,54.126322,0.94,17.492514,0.94,17.710098,0.68,7.696862,0.88,0.86
3,BRITANNIA.NS,4959.00,N/A,53.399569,0.92,12.561500,0.86,18.460032,0.72,6.711506,0.80,0.825
4,ITC.NS,447.60,N/A,72.472266,0.98,34.068733,0.98,17.880501,0.70,4.543855,0.54,0.8
5,NESTLEIND.NS,22913.00,N/A,36.758099,0.78,15.729710,0.90,26.949994,0.94,4.324449,0.50,0.78
6,TITAN.NS,2907.40,N/A,38.587417,0.84,15.235664,0.88,21.172520,0.80,4.587073,0.56,0.77
7,M&M.NS,1390.55,N/A,35.812439,0.76,7.125380,0.64,18.813294,0.78,7.233695,0.84,0.755
8,TATACONSUM.NS,864.70,N/A,18.196075,0.34,8.335390,0.74,26.825918,0.92,10.806478,0.98,0.745
9,BAJAJ-AUTO.NS,4729.00,N/A,32.712957,0.68,30.693784,0.96,26.815019,0.90,3.629063,0.42,0.74


## **Calculating the Number of Shares to Buy**




We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [19]:
portfolio_input()

Enter the value of your portfolio:2000000


In [20]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,TATAMOTORS.NS,570.95,318,37.720352,0.82,36.861044,1.00,38.641055,1.00,7.431480,0.86,0.92
1,INDUSINDBK.NS,1294.75,140,59.913329,0.96,7.919223,0.66,28.095639,0.96,9.693465,0.96,0.885
2,ULTRACEMCO.NS,8385.05,21,54.126322,0.94,17.492514,0.94,17.710098,0.68,7.696862,0.88,0.86
3,BRITANNIA.NS,4959.00,36,53.399569,0.92,12.561500,0.86,18.460032,0.72,6.711506,0.80,0.825
4,ITC.NS,447.60,406,72.472266,0.98,34.068733,0.98,17.880501,0.70,4.543855,0.54,0.8
5,NESTLEIND.NS,22913.00,7,36.758099,0.78,15.729710,0.90,26.949994,0.94,4.324449,0.50,0.78
6,TITAN.NS,2907.40,62,38.587417,0.84,15.235664,0.88,21.172520,0.80,4.587073,0.56,0.77
7,M&M.NS,1390.55,130,35.812439,0.76,7.125380,0.64,18.813294,0.78,7.233695,0.84,0.755
8,TATACONSUM.NS,864.70,210,18.196075,0.34,8.335390,0.74,26.825918,0.92,10.806478,0.98,0.745
9,BAJAJ-AUTO.NS,4729.00,38,32.712957,0.68,30.693784,0.96,26.815019,0.90,3.629063,0.42,0.74


## **Formatting Our Excel Output**
We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization.

In [21]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## **Creating the Formats We'll Need For Our .xlsx File**
We'll need four main formats for our Excel document:

*   String format for tickers
*   ₹XX.XX format for stock prices

*   ₹XX,XXX format for market capitalization
*   Integer format for the number of shares to purchase


*   Float formats with 3 decimal for each valuation metric


In [ ]:
string_template = writer.book.add_format(
        {
            'font_color': '#ffffff',
            'bg_color': '#0a0a23',
            'border': 1
        }
    )

INR_template = writer.book.add_format(
        {
            'num_format':'₹0.00',
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'border': 1
        }
    )
float_template = writer.book.add_format(
        {
            'num_format':'0.000',
            'border': 1
        }
    )
percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'border': 1
        }
    )

In [ ]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', INR_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', float_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## **Saving Our Excel Output**
Saving our Excel output is very easy:

In [ ]:
writer.save()

<ipython-input-135-16d93d5523a4>:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
